# Sentinel-2 Validation Data Analysis - Point-based and Parallel  <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a)

## Background
TBA

## Description
This notebook explains how you can perform validation analysis for S2 SCL layer using collected ground truth dataset and window-based sampling. 

The notebook demonstrates how to:

1. Load validation points for each partner institutions following cleaning stage as an ESRI shapefile
2. Query Sentinel-2 SCL layer for validation points and capture available Sentinel-2 observation available
3. Extract statistics for each S2 observation in each validation point using multiprocessing functionality 
4. Extract a LUT for each point that contains both validation info and S2 result for each month 
***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages
Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

import time 
import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray as xr
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from rasterio.mask import mask
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
from deafrica_plotting import map_shapefile,display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard,deepcopy
from deafrica_dask import create_local_dask_cluster

#for parallelisation 
from multiprocessing import Pool, Manager
import multiprocessing as mp
from tqdm import tqdm

### Analysis parameters

To analyse validation points collected by each partner institution, we need to obtain WOfS surface water observation data that corresponds with the labelled input data locations. 

### Loading Dataset

1. Load validation points for each partner institutions as a list of observations each has a location and month
    * Load the cleaned validation file as ESRI `shapefile`
    * Inspect the shapefile

In [2]:
#generate query object 
query ={'resolution':(-20, 20),
        'group_by':'solar_day',
        'output_crs':'EPSG:6933'}

In [3]:
path = '../Supplementary_data/Validation/Refined/groundtruth_RCMRD.shp'

In [4]:
input_data = gpd.read_file(path).to_crs('epsg:6933') #reading the table and converting CRS to metric 
input_data.columns

Index(['Unnamed_ 0', 'Unnamed__1', 'PLOT_ID', 'LON', 'LAT', 'FLAGGED',
       'ANALYSES', 'SENTINEL2Y', 'WATER', 'NO_WATER', 'BAD_IMAGE', 'NOT_SURE',
       'CLASS', 'COMMENT', 'MONTH', 'WATERFLAG', 'geometry'],
      dtype='object')

In [5]:
input_data= input_data.drop(['Unnamed_ 0'], axis=1)

In [6]:
coords = [(x,y) for x, y in zip(input_data.geometry.x, input_data.geometry.y)]

### Sample S2 at the ground truth coordinates 

In [7]:
#function to sample WOfS for each validation point for early five days of each month 
def get_S2_for_point(index, row, input_data, query, results_wet, results_clear):
    dc = datacube.Datacube(app='S2_accuracy')
    #get the month value for each index
    month = input_data.loc[index]['MONTH'] 
    #set the time for query of WOfS database according to the first five days before and after of each calendar month
    time = '2018-'+f'{month:02d}'    
    plot_id = input_data.loc[index]['PLOT_ID']
    #having the original query as it is 
    dc_query = deepcopy(query) 
    geom = geometry.Geometry(input_data.geometry.values[index].__geo_interface__, 
                             geometry.CRS('EPSG:6933'))
    q = {"geopolygon":geom}
    t = {"time":time} 
    
    #updating the query
    dc_query.update(t)
    dc_query.update(q)
    
    ds = dc.load(product ="s2_l2a",
                 measurements=['SCL'],
                 **dc_query)
    
    #Check if water is observed by SCL 
    if np.any(ds.SCL.values==6) == True:
        results_wet.update({str(int(plot_id))+"_"+str(month) : 1})
    else:
        results_wet.update({str(int(plot_id))+"_"+str(month) : 0})
    
    #number of clear 
    n_clear = np.count_nonzero(ds.SCL.isin([2,4,5,6,7]).values)
    results_clear.update({str(int(plot_id))+"_"+str(month) : int(n_clear)})
                          
#     wet_nocloud = (ds.SCL == 6).astype(int)
#     ds.SCL.values.
#     print(xr.ufuncs.isnan(ds.where(wet_nocloud)))
    
#     S2_wet = wet_nocloud.sum() #this returns one value for a month 
#     print(S2_wet)
#     # Define a mask for dry and clear pixels 
#     dry_nocloud  = (ds.SCL.isin([2,4,5,7])).astype(int)
#     S2_dry = dry_nocloud.sum() #this returns one value for a month 
#     #Define a mask for dry and clear pixels 
#     clear = (S2_wet | S2_dry)#.all().values  #My major doubt is in here 
#     n_clear = clear.sum() #record this and use it to filter out month with no valid data 
#     if n_clear > 0:
#         wet = S2_wet.isel(time=clear).max().values  #record this as S2 has seen water on the day 
#     else:
#         wet = 0 

#     results_wet.update({str(int(plot_id))+"_"+str(month) : int(wet)})
#     results_clear.update({str(int(plot_id))+"_"+str(month) : int(n_clear)})


### Testing For-Loop 

In [ ]:
results_wet_test = dict()
results_clear_test = dict()

for index, row in input_data.iterrows():
    get_S2_for_point(index, row, input_data, query, results_wet_test, results_clear_test)


### Parallel Processing 

In [8]:
def _parallel_fun(input_data, query, ncpus):
    
    manager = mp.Manager()
    results_wet = manager.dict()
    results_clear = manager.dict()
   
    # progress bar
    pbar = tqdm(total=len(input_data))
        
    def update(*a):
        pbar.update()

    with mp.Pool(ncpus) as pool:
        for index, row in input_data.iterrows():
            pool.apply_async(get_S2_for_point,
                                 [index,
                                 row,
                                 input_data,
                                 query,
                                 results_wet,
                                 results_clear], callback=update)
        pool.close()
        pool.join()
        pbar.close()
        
    return results_wet, results_clear

### Parallel for runnning

In [9]:
# %%time
wet, clear = _parallel_fun(input_data, query, ncpus=2)

  1%|          | 70/8677 [00:54<2:30:13,  1.05s/it]

FileNotFoundError: [Errno 2] No such file or directory

In [13]:
wetdf = pd.DataFrame.from_dict(wet, orient = 'index')
cleardf = pd.DataFrame.from_dict(clear,orient='index')
df2 = wetdf.merge(cleardf, left_index=True, right_index=True)
df2 = df2.rename(columns={'0_x':'CLASS_WET','0_y':'CLEAR_OBS'})
#split the index (which is plotid+month) into seperate columns
for index, row in df2.iterrows():
    df2.at[index,'PLOT_ID'] = index.split('_')[0] +'.0'
    df2.at[index,'MONTH'] = index.split('_')[1]
#reset the index
df2 = df2.reset_index(drop=True)
#convert plot id and month to str to help with matching
input_data['PLOT_ID'] = input_data.PLOT_ID.astype(str)
input_data['MONTH']= input_data.MONTH.astype(str)
# merge both dataframe at locations where plotid and month match
final_df = pd.merge(input_data, df2, on=['PLOT_ID','MONTH'], how='outer')

In [14]:
# As water flag more than 1 and also clear observation equal to zero 
indexNames = final_df[(final_df['WATERFLAG'] > 1) | (final_df['CLEAR_OBS'] == 0) | (final_df['CLEAR_OBS'].isna()) ].index 
final_df.drop(indexNames, inplace=True)

In [15]:
final_df.shape

(23, 18)

In [16]:
final_df

,Unnamed__1,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2Y,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG,geometry,CLASS_WET,CLEAR_OBS
0,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,1,1,POINT (2882610.000 277890.000),0.0,5.0
1,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,2,1,POINT (2882610.000 277890.000),0.0,2.0
2,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,3,1,POINT (2882610.000 277890.000),0.0,4.0
3,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,4,1,POINT (2882610.000 277890.000),1.0,2.0
4,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,5,1,POINT (2882610.000 277890.000),1.0,5.0
5,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,6,1,POINT (2882610.000 277890.000),1.0,3.0
6,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,7,1,POINT (2882610.000 277890.000),1.0,4.0
7,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,8,1,POINT (2882610.000 277890.000),0.0,2.0
8,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,9,1,POINT (2882610.000 277890.000),0.0,3.0
9,0,137387037.0,29.875854,2.178788,0.0,1.0,2018,1-12,0,10,0,Open water - freshwater,Point is within the river channel,10,1,POINT (2882610.000 277890.000),0.0,3.0


In [ ]:
final_df.to_csv(('../Supplementary_data/Validation/Refined/RCMRD_points.csv'))

In [ ]:
print(datacube.__version__)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** September 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)